# 임신 성공 확률 예측 🕵️
### 제공된 데이터를 분석하고 🧐, 임신 성공 여부 확률 Model 을 학습합니다 📖.

## Table of Contents
1. Library Import
2. Load Data
3. 데이터 보정  
    3-1. function  
    3-2. 보정 적용  
    3-3. 추가 파생 변수  
4. 데이터 매핑
5. Model Training
6. Inference


## 1. Library Import
- (Kaggle 환경을 사용하였습니다. 필수 라이브러리 목록은 하단과 같습니다.)
- Python version: 3.10.12
- numpy==1.26.4
- pandas==2.2.2
- scikit-learn==1.2.2
- xgboost==2.0.3
- lightgbm==4.5.0
- catboost==1.2.7
- scipy==1.13.1
- pip install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.2.2 xgboost==2.0.3 lightgbm==4.5.0 catboost==1.2.7 scipy==1.13.1

In [1]:
# basic
import os
import random

import numpy as np
import pandas as pd

# scikit-learn 관련
from sklearn.linear_model import LogisticRegression

# xgboost
from xgboost import XGBClassifier

# lightgbm
from lightgbm import LGBMClassifier

# catboost
from catboost import CatBoostClassifier, Pool

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

## 2. Load Data
- Data 위치에 맞게 재설정 해주시면 감사하겠습니다.

In [ ]:
# load data
train = pd.read_csv('./data/train.csv').drop(columns='ID')
test = pd.read_csv('./data/test.csv').drop(columns='ID')
df_sub = pd.read_csv('./data/sample_submission.csv')

## 3. 데이터 보정
- 데이터 안에서 어긋나있는 내용을 조절하기 위해 적용한 구간입니다.
- 현재 조정된 값은 다음과 같습니다.
    - 난자 수
    - 배아 수
    - 정자 출처

### 3-1. 함수 모음

- 혼합 된 난자의 수가 맞지 않은 구간에서 해결되지 않은 값을 받기 위해 미리 '확인되지 않은 혼합된 난자 수' 칼럼을 생성하였습니다.

In [3]:
# 미할당 정자 수 칼럼 생성
train['확인되지 않은 혼합된 난자 수'] = 0
test['확인되지 않은 혼합된 난자 수'] = 0

In [4]:
# 혼합된 난자 수 보정하기
def nanja_count_settings(data):
    cri = [
        (data['혼합된 난자 수'] == data['파트너 정자와 혼합된 난자 수'] + data['기증자 정자와 혼합된 난자 수'] + data['확인되지 않은 혼합된 난자 수']) & ~(data['혼합된 난자 수'].isna()),
        (data['혼합된 난자 수'] > data['파트너 정자와 혼합된 난자 수'] + data['기증자 정자와 혼합된 난자 수'] + data['확인되지 않은 혼합된 난자 수']) & (data['정자 출처'] == '배우자 제공') & ~(data['혼합된 난자 수'].isna()),
        (data['혼합된 난자 수'] > data['파트너 정자와 혼합된 난자 수'] + data['기증자 정자와 혼합된 난자 수'] + data['확인되지 않은 혼합된 난자 수']) & (data['정자 출처'] == '기증 제공') & ~(data['혼합된 난자 수'].isna()),
        (data['혼합된 난자 수'] > data['파트너 정자와 혼합된 난자 수'] + data['기증자 정자와 혼합된 난자 수'] + data['확인되지 않은 혼합된 난자 수']) & (data['정자 출처'] == '미할당') & ~(data['혼합된 난자 수'].isna()),
    ]
    con1 = [
        data['파트너 정자와 혼합된 난자 수'],
        data['혼합된 난자 수'],
        0,
        0
    ]
    con2 = [
        data['기증자 정자와 혼합된 난자 수'],
        0,
        data['혼합된 난자 수'],
        0
    ]
    con3 = [
        0, 0, 0, data['혼합된 난자 수']
    ]
    data['파트너 정자와 혼합된 난자 수'] = np.select(cri, con1, default = data['파트너 정자와 혼합된 난자 수'])
    data['기증자 정자와 혼합된 난자 수'] = np.select(cri, con2, default = data['기증자 정자와 혼합된 난자 수'])
    data['확인되지 않은 혼합된 난자 수'] = np.select(cri, con3, default = data['확인되지 않은 혼합된 난자 수'])

    return data

In [5]:
# 수집 + 해동 = 저장 + 혼합 맞추기
# 수집 + 해동이 더 작은 경우 수집에 몰빵하기
def collect_nanja(data):
    cri = [
        data['해동 난자 수'] + data['수집된 신선 난자 수'] < data['저장된 신선 난자 수'] + data['혼합된 난자 수']
    ]
    con = [
        data['저장된 신선 난자 수'] + data['혼합된 난자 수']
    ]
    data['수집된 신선 난자 수'] = np.select(cri, con, default = data['수집된 신선 난자 수'])

    return data

In [6]:
# 배아 수 고치기
def egg_count_adj(data):

    data['총 생성 배아 수'] = np.where(data['미세주입된 난자 수'] < data['미세주입에서 생성된 배아 수'], data['미세주입된 난자 수'], data['총 생성 배아 수'])
    data['미세주입에서 생성된 배아 수'] = np.where(data['미세주입된 난자 수'] < data['미세주입에서 생성된 배아 수'], data['미세주입된 난자 수'], data['미세주입에서 생성된 배아 수'])

    return data

In [7]:
# 생성 + 해동 = 저장 + 혼합 맞추기
# 생성 + 해동이 더 작은 경우 수집에 몰빵하기
def collect_baea(data):
    cri = [
        (data['총 생성 배아 수'] + data['해동된 배아 수'] < data['저장된 배아 수'] + data['이식된 배아 수'])
    ]

    con = [
        data['저장된 배아 수'] + data['이식된 배아 수']
    ]

    data['총 생성 배아 수'] = np.select(cri, con, default = data['총 생성 배아 수'])

    return data

In [8]:
# PGS 관련 변수 합쳐놓기
def pgs_adj(data):

    data['PGS 시술 여부'][data['PGS 시술 여부'].isna()] = 0
    data['착상 전 유전 검사 사용 여부'][data['착상 전 유전 검사 사용 여부'].isna()] = 0

    cri = [
        (data['착상 전 유전 검사 사용 여부'] == 0) & (data['PGS 시술 여부'] == 0),
        (data['착상 전 유전 검사 사용 여부'] == 1) & (data['PGS 시술 여부'] == 0),
        (data['착상 전 유전 검사 사용 여부'] == 1) & (data['PGS 시술 여부'] == 1),
    ]

    con = [
        0, -1, 1
    ]
    
    data['PGS'] = np.select(cri, con, default = -999)

    return data

In [9]:
# PGD 관련 변수 합쳐놓기
def pgd_adj(data):

    data['PGD 시술 여부'][data['PGD 시술 여부'].isna()] = 0
    data['착상 전 유전 진단 사용 여부'][data['착상 전 유전 진단 사용 여부'].isna()] = 0

    cri = [
        (data['착상 전 유전 진단 사용 여부'] == 0) & (data['PGD 시술 여부'] == 0),
        (data['착상 전 유전 진단 사용 여부'] == 1) & (data['PGD 시술 여부'] == 0),
        (data['착상 전 유전 진단 사용 여부'] == 1) & (data['PGD 시술 여부'] == 1),
    ]

    con = [
        0, -1, 1
    ]
    
    data['PGD'] = np.select(cri, con, default = -999)

    return data

In [10]:
# DI와 IVF
def sisul_type_adj(data):

    cri = [
        (data['시술 유형'] == 'DI'),
        (data['시술 유형'] == 'IVF') & (data['난자 출처'] == '기증 제공'),
        (data['시술 유형'] == 'IVF') & (data['정자 출처'].isin(['기증 제공', '배우자 및 기증 제공'])),
    ]

    con = [
        1, 1, 1
    ]

    data['DI시술방법'] = np.select(cri, con, default=0)
    data['IVF시술방법'] = np.where(data['시술 유형'] == 'IVF', 1, 0)

    return data

In [11]:
# 난자출처에 따른 해당 주기의 난자 주인 나이를 설명
def nanja_own_age(data):
    cri = [
        data['난자 출처'] == '본인 제공',
        data['난자 출처'] == '알 수 없음',
        data['난자 출처'] == '기증 제공',
    ]

    con = [
        data['시술 당시 나이'],
        data['시술 당시 나이'],
        data['난자 기증자 나이']
    ]

    data['해당 주기 난자 주인 나이'] = np.select(cri, con, default = np.nan)

    return data

In [12]:
# 해당 주기 난자 주인 나이 매핑, 나이가 많을수록 낮은 값을 책정
def nanja_level_mapping(data):
    cri = [
        data['해당 주기 난자 주인 나이'] == '만18-34세',
        data['해당 주기 난자 주인 나이'] == '만45-50세',
        data['해당 주기 난자 주인 나이'] == '만35-37세',
        data['해당 주기 난자 주인 나이'] == '만38-39세',
        data['해당 주기 난자 주인 나이'] == '만21-25세',
        data['해당 주기 난자 주인 나이'] == '만40-42세',
        data['해당 주기 난자 주인 나이'] == '만43-44세',
        data['해당 주기 난자 주인 나이'] == '만31-35세',
        data['해당 주기 난자 주인 나이'] == '만26-30세',
        data['해당 주기 난자 주인 나이'] == '만20세 이하',
        data['해당 주기 난자 주인 나이'] == '알 수 없음'
    ]

    con = [
        6, 1, 5, 4, 6, 3, 2, 6, 6, 6, 0
    ]
    data['해당 주기 난자 주인 나이'] = np.select(cri, con, default = data['해당 주기 난자 주인 나이'])
    return data

### 3-2. 보정 함수 적용

- 값이 자연스럽지 않은 구성에 대해 함수 보정을 적용하였습니다.

In [13]:
# 혼합된 난자 불일치 변경
train = nanja_count_settings(train)
test = nanja_count_settings(test)

# 해동, 수집, 저장, 혼합에 관해 수식에 맞는 값으로 적용
train = collect_nanja(train)
test = collect_nanja(test)

# 총 배아수를 넘는 숫자 보정
train = egg_count_adj(train)
test = egg_count_adj(test)

# 생성, 해동, 이식, 저장 배아수 조정
train = collect_baea(train)
test = collect_baea(test)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


In [14]:
# 정자의 출처가 둘다 혼합하는 경우에는 배우자 및 기증 제공으로 이동
train['정자 출처'] = np.where((train['파트너 정자와 혼합된 난자 수'] > 0) & (train['기증자 정자와 혼합된 난자 수'] > 0), '배우자 및 기증 제공', train['정자 출처'])
test['정자 출처'] = np.where((test['파트너 정자와 혼합된 난자 수'] > 0) & (test['기증자 정자와 혼합된 난자 수'] > 0), '배우자 및 기증 제공', test['정자 출처'])

### 3-3. 추가 파생변수

In [15]:
# ICSI를 적용하지 않은 배아에 대한 파생 변수
train['IVF에서 생성된 배아 수'] = train['총 생성 배아 수'] - train['미세주입에서 생성된 배아 수']
test['IVF에서 생성된 배아 수'] = test['총 생성 배아 수'] - test['미세주입에서 생성된 배아 수']

train['IVF 배아 이식 수'] = train['이식된 배아 수'] - train['미세주입 배아 이식 수']
test['IVF 배아 이식 수'] = test['이식된 배아 수'] - test['미세주입 배아 이식 수']

train['IVF 후 저장된 배아 수'] = train['저장된 배아 수'] - train['미세주입 후 저장된 배아 수']
test['IVF 후 저장된 배아 수'] = test['저장된 배아 수'] - test['미세주입 후 저장된 배아 수']

# 폐기된 배아 수
train['폐기된 배아 수'] = train['총 생성 배아 수'] + train['해동된 배아 수'] - (train['저장된 배아 수'] + train['이식된 배아 수'])
test['폐기된 배아 수'] = test['총 생성 배아 수'] + test['해동된 배아 수'] - (test['저장된 배아 수'] + test['이식된 배아 수'])

# 배아 생성 실패 난자 수
train['미세주입 배아 실패 수'] = train['미세주입된 난자 수'] - train['미세주입에서 생성된 배아 수']
test['미세주입 배아 실패 수'] = test['미세주입된 난자 수'] - test['미세주입에서 생성된 배아 수']

train['IVF 배아 실패 수'] = train['혼합된 난자 수'] - train['미세주입된 난자 수'] - train['IVF에서 생성된 배아 수']
test['IVF 배아 실패 수'] = test['혼합된 난자 수'] - test['미세주입된 난자 수'] - test['IVF에서 생성된 배아 수']

In [16]:
# PGS 변수 합치기
train = pgs_adj(train)
test = pgs_adj(test)

# PGD 변수 합치기
train = pgd_adj(train)
test = pgd_adj(test)

# 시술 타입 대분류 공통적인 특징 보정
train = sisul_type_adj(train)
test = sisul_type_adj(test)

# 난자의 주인에 대한 나이 생성 (본인난자 사용하면 본인, 기증자 난자 사용하면 기증자 나이)
train = nanja_own_age(train)
test = nanja_own_age(test)

# 해당 주기의 난자 주인 나이를 일치하기 위한 매핑 방법
train = nanja_level_mapping(train)
test = nanja_level_mapping(test)

# 싹다 -999로 채워 EDA 하는 경우에 -999를 보면 Missing임을 알수있도록
train = train.fillna('-999')
test = test.fillna('-999')

# 겹치는 Column과 상수 Column 제거
constant_columns = train.nunique() == 1
columns_to_drop = train.columns[constant_columns]

train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)

# 배란 유도 유형의 종류가 단 1건씩 두 종류만 존재해 배제 결정
train = train.drop(columns='배란 유도 유형')
test = test.drop(columns='배란 유도 유형')

<ipython-input-8-7c69353544e7>:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['PGS 시술 여부'][data['PGS 시술 여부'].isna()] = 0
<ipython-input-8-7c69353544e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

## 4. 데이터 Mapping

- 나이, 출산횟수와 같은 변수의 경우 범주형 데이터로 기록됨  
- 임의로 아무 숫자나 반영하지 않게 적용하기 위해 각 데이터별로 매핑 결정

### 4-1. Mapping

In [ ]:
# 나이 컬럼 처리
train['정자 기증자 나이'] = train['정자 기증자 나이'].replace("-999", "알 수 없음")
test['정자 기증자 나이'] = test['정자 기증자 나이'].replace("-999", "알 수 없음")
train['난자 기증자 나이'] = train['난자 기증자 나이'].replace("-999", "알 수 없음")
test['난자 기증자 나이'] = test['난자 기증자 나이'].replace("-999", "알 수 없음")

# 나이가 높을수록 낮은 값을 책정
age_mapping = {
     "만18-34세" : 1024,
     "만35-37세" : 512,
     "만38-39세" : 256,
     "만40-42세" : 128,
     "만43-44세" : 32,
     "만45-50세" : 4,
     "알 수 없음" : -999,
}
train["시술 당시 나이"] = train["시술 당시 나이"].map(age_mapping)
test["시술 당시 나이"] = test["시술 당시 나이"].map(age_mapping)

age_mapping = {
     "만20세 이하" : 1024,
     "만21-25세" : 512,
     "만26-30세" : 128,
     "만31-35세" : 16,
     "알 수 없음" : -999,
}

train["난자 기증자 나이"] = train["난자 기증자 나이"].map(age_mapping)
test["난자 기증자 나이"] = test["난자 기증자 나이"].map(age_mapping)

age_mapping = {
     "만20세 이하" : 1024,
     "만21-25세" : 512,
     "만26-30세" : 256,
     "만31-35세" : 128,
     "만36-40세" : 16,
     "만41-45세" : 4,
     "알 수 없음" : -999,
}

train["정자 기증자 나이"] = train["정자 기증자 나이"].map(age_mapping)
test["정자 기증자 나이"] = test["정자 기증자 나이"].map(age_mapping)

# 시술 유형 처리
mapping = {
     "DI" : -100,
     "IVF" : 100
}
train["시술 유형"] = train["시술 유형"].map(mapping)
test["시술 유형"] = test["시술 유형"].map(mapping)


# 횟수 처리
mapping = {
    "0회": 0, "1회": 1, "2회": 2, "3회": 3, "4회": 4, "5회": 5,
    "6회 이상": 20, '-999' : -999
}
# 6회 이상이라는 말에 의미를 담기 위해서 20로 매핑함.

# 클리닉 횟수 관련
train["총 시술 횟수"] = train["총 시술 횟수"].map(mapping)
test["총 시술 횟수"] = test["총 시술 횟수"].map(mapping)

train["DI 시술 횟수"] = train["DI 시술 횟수"].map(mapping)
test["DI 시술 횟수"] = test["DI 시술 횟수"].map(mapping)

train["IVF 시술 횟수"] = train["IVF 시술 횟수"].map(mapping)
test["IVF 시술 횟수"] = test["IVF 시술 횟수"].map(mapping)

train["클리닉 내 총 시술 횟수"] = train["클리닉 내 총 시술 횟수"].map(mapping)
test["클리닉 내 총 시술 횟수"] = test["클리닉 내 총 시술 횟수"].map(mapping)

# 임신 횟수 관련 
train["총 임신 횟수"] = train["총 임신 횟수"].map(mapping)
test["총 임신 횟수"] = test["총 임신 횟수"].map(mapping)

train["IVF 임신 횟수"] = train["IVF 임신 횟수"].map(mapping)
test["IVF 임신 횟수"] = test["IVF 임신 횟수"].map(mapping)

train["DI 임신 횟수"] = train["DI 임신 횟수"].map(mapping)
test["DI 임신 횟수"] = test["DI 임신 횟수"].map(mapping)

# 출산 횟수 관련 
train["총 출산 횟수"] = train["총 출산 횟수"].map(mapping)
test["총 출산 횟수"] = test["총 출산 횟수"].map(mapping)

train["IVF 출산 횟수"] = train["IVF 출산 횟수"].map(mapping)
test["IVF 출산 횟수"] = test["IVF 출산 횟수"].map(mapping)

train["DI 출산 횟수"] = train["DI 출산 횟수"].map(mapping)
test["DI 출산 횟수"] = test["DI 출산 횟수"].map(mapping)

# 코드가 뒤로 갈수록 나이가 많은 분들이 시술을 받는 경향을 받아, 선형적으로 구성, 코드가 매핑상 뒤로갈수록 나이가 많은 사람이 임신을 하는 경우가 많은 쪽으로 배치
# EDA상 코드 시기상으로도 이게 맞다는 사실을 알게 됨.
code_to_year = {
    'TRXQMD' : 1,
    'TRZKPL' : 2,
    'TRVNRY' : 3,
    'TRJXFG' : 4,
    'TRYBLT' : 5,
    'TRCMWS' : 6,
    'TRDQAZ' : 7,
}

train['시술 시기 코드'] = train['시술 시기 코드'].map(code_to_year)
test['시술 시기 코드'] = test['시술 시기 코드'].map(code_to_year)

# 난자 출처 매핑
mapping = {
     "본인 제공" : 1,
     "기증 제공" : -1,
     "알 수 없음" : 1, # DI의 경우 99프로 이상이 본인의 난자와 수정하는 과정임.
}
train['난자 출처'] = train['난자 출처'].map(mapping)
test['난자 출처'] = test['난자 출처'].map(mapping)

# 정자 출처 매핑
mapping = {
     "배우자 제공" : 1,
     "기증 제공" : -1,
     "배우자 및 기증 제공" : 0,
     "미할당" : -999,
}

train['정자 출처'] = train['정자 출처'].map(mapping)
test['정자 출처'] = test['정자 출처'].map(mapping)

### 4-2. Encoding 및 데이터 형식 변경

In [ ]:
# 특정 시술 유형의 :를 _로 바꿔서, 오류를 없앰.
train['특정 시술 유형'] =train['특정 시술 유형'].str.replace(r'\s+', '', regex=True).str.replace(":", "_")
test['특정 시술 유형'] =test['특정 시술 유형'].str.replace(r'\s+', '', regex=True).str.replace(":", "_")

# 특정 시술 유형 onehot
for i in ['ICSI', 'IUI', 'ICI', 'GIFT', 'FER', 'GenericDI', 'IVI', 'BLASTOCYST', 'AH', 'Unknown', 'IVF', '-999', 'ICSI_ICSI', 'IVF_IVF', 'ICSI_Unknown', 'BLASTOCYST_IVF', 'IVF_Unknown', 'ICSI/BLASTOCYST', 'IVF/BLASTOCYST', 'ICSI/AH', 'IVF/AH', 'ICSI/BLASTOCYST_IVF/BLASTOCYST']:
    train['특시유_' + i] = np.where(train['특정 시술 유형'].str.contains(i, regex=False), 1, 0)
    test['특시유_' + i] = np.where(test['특정 시술 유형'].str.contains(i, regex=False), 1, 0)

# 배아 생성 주요 이유 onehot
for i in ['-999', '기증용', '난자 저장용', '배아 저장용', '연구용', '현재 시술용']:
    train['배생이_' + i] = np.where(train['배아 생성 주요 이유'].str.contains(i), 1, 0)
    test['배생이_' + i] = np.where(test['배아 생성 주요 이유'].str.contains(i), 1, 0)

train = train.drop(columns = ['배아 생성 주요 이유'])
test = test.drop(columns = ['배아 생성 주요 이유'])

# Int화
train['착상 전 유전 진단 사용 여부'] = train['착상 전 유전 진단 사용 여부'].astype(int)
test['착상 전 유전 진단 사용 여부'] = test['착상 전 유전 진단 사용 여부'].astype(int)
train['착상 전 유전 검사 사용 여부'] = train['착상 전 유전 검사 사용 여부'].astype(int)
test['착상 전 유전 검사 사용 여부'] = test['착상 전 유전 검사 사용 여부'].astype(int)
train['임신 시도 또는 마지막 임신 경과 연수'] = train['임신 시도 또는 마지막 임신 경과 연수'].astype(int)
test['임신 시도 또는 마지막 임신 경과 연수'] = test['임신 시도 또는 마지막 임신 경과 연수'].astype(int)
train['단일 배아 이식 여부'] = train['단일 배아 이식 여부'].astype(int)
test['단일 배아 이식 여부'] = test['단일 배아 이식 여부'].astype(int)

# 시술유형에 따른 결측치 대체매핑 -> DI인 경우와 IVF인 경우에 의미가 다름. 그거에 맞게 결측에 대한 값을 재매핑
train.loc[(train['시술 유형'] == 100) & (train['착상 전 유전 검사 사용 여부'] == '-999'), '착상 전 유전 검사 사용 여부'] = 0
test.loc[(test['시술 유형'] == 100) & (test['착상 전 유전 검사 사용 여부'] == '-999'), '착상 전 유전 검사 사용 여부'] = 0

train.loc[(train['시술 유형'] == 100) & (train['PGD 시술 여부'] == '-999'), 'PGD 시술 여부'] = 0
train.loc[(train['시술 유형'] == 100) & (train['PGS 시술 여부'] == '-999'), 'PGS 시술 여부'] = 0
test.loc[(test['시술 유형'] == 100) & (test['PGD 시술 여부'] == '-999'), 'PGD 시술 여부'] = 0
test.loc[(test['시술 유형'] == 100) & (test['PGS 시술 여부'] == '-999'), 'PGS 시술 여부'] = 0

test.loc[(test['시술 유형'] == 100) & (test['난자 채취 경과일'] == '-999'), '난자 채취 경과일'] = 0
test.loc[(test['시술 유형'] == 100) & (test['난자 해동 경과일'] == '-999'), '난자 해동 경과일'] = 0
test.loc[(test['시술 유형'] == 100) & (test['난자 혼합 경과일'] == '-999'), '난자 혼합 경과일'] = 0
test.loc[(test['시술 유형'] == 100) & (test['배아 이식 경과일'] == '-999'), '배아 이식 경과일'] = 0
test.loc[(test['시술 유형'] == 100) & (test['배아 해동 경과일'] == '-999'), '배아 해동 경과일'] = 0

train.loc[(train['시술 유형'] == 100) & (train['난자 채취 경과일'] == '-999'), '난자 채취 경과일'] = 0
train.loc[(train['시술 유형'] == 100) & (train['난자 해동 경과일'] == '-999'), '난자 해동 경과일'] = 0
train.loc[(train['시술 유형'] == 100) & (train['난자 혼합 경과일'] == '-999'), '난자 혼합 경과일'] = 0
train.loc[(train['시술 유형'] == 100) & (train['배아 이식 경과일'] == '-999'), '배아 이식 경과일'] = 0
train.loc[(train['시술 유형'] == 100) & (train['배아 해동 경과일'] == '-999'), '배아 해동 경과일'] = 0

# str인 것들을 다 int화
col = [
    "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "이식된 배아 수",
    "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수",
    "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수", "동결 배아 사용 여부",
    "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부",
    "PGS 시술 여부", "난자 채취 경과일", "난자 해동 경과일", "난자 혼합 경과일",
    "배아 이식 경과일", "배아 해동 경과일",  'IVF에서 생성된 배아 수', 'IVF 배아 이식 수',
    'IVF 후 저장된 배아 수', '폐기된 배아 수', '미세주입 배아 실패 수', 'IVF 배아 실패 수']
train[col] = train[col].astype(int)
test[col] = test[col].astype(int)

### 4-3. 생성 컬럼

In [ ]:
# 횟수로부터, 경험의 여부 변수를 도축
train['임신 경험 여부'] = train['총 임신 횟수'].apply(lambda x: 1 if x > 0 else 0)
train['출산 경험 여부'] = train['총 출산 횟수'].apply(lambda x: 1 if x > 0 else 0)
train['시술 경험 여부'] = train['총 시술 횟수'].apply(lambda x: 1 if x > 0 else 0)

test['임신 경험 여부'] = test['총 임신 횟수'].apply(lambda x: 1 if x > 0 else 0)
test['출산 경험 여부'] = test['총 출산 횟수'].apply(lambda x: 1 if x > 0 else 0)
test['시술 경험 여부'] = test['총 시술 횟수'].apply(lambda x: 1 if x > 0 else 0)

# 나이와 이식수를 곱하므로, 그 교호작용을 꾀함
train['나이_X_이식수'] = train['시술 당시 나이'] * train['이식된 배아 수']
test['나이_X_이식수'] = test['시술 당시 나이'] * test['이식된 배아 수']

# 배란자극이 된 배아가 이식되었을 때, 그 효능을 보고자 함.
train['배란자극_X_이식배아수'] = train['배란 자극 여부'].astype(int) * train['이식된 배아 수']
test['배란자극_X_이식배아수'] = test['배란 자극 여부'].astype(int) * test['이식된 배아 수']

# 남은 배아수
train['잉여_배아_수'] = train['총 생성 배아 수'] - train['이식된 배아 수']
test['잉여_배아_수'] = test['총 생성 배아 수'] - test['이식된 배아 수']

# 실제 1만 이식하는 경우
train['단일_배아_이식_여부_수정'] = train['이식된 배아 수'].apply(lambda x: 1 if x == 1 else 0)
test['단일_배아_이식_여부_수정'] = test['이식된 배아 수'].apply(lambda x: 1 if x == 1 else 0)

# 클리닉 외에 시술이 있는지
train['클리닉 외 시술 여부'] = (train['총 시술 횟수'] - train['클리닉 내 총 시술 횟수']) > 0
test['클리닉 외 시술 여부'] = (test['총 시술 횟수'] - train['클리닉 내 총 시술 횟수']) > 0

# 생성된 배아수와 이식된 배아수의 비율
train['이식률'] = np.where(
    train['총 생성 배아 수'] > 0,
    train['이식된 배아 수'] / train['총 생성 배아 수'] * 100,
-999)
test['이식률'] = np.where(
    test['총 생성 배아 수'] > 0,
    test['이식된 배아 수'] / test['총 생성 배아 수'] * 100,
-999)

# 미세주입된 난자와 그로부터 생성된 배아수의 비율
train['수정률'] = np.where(
    train['미세주입된 난자 수'] > 0,
    train['미세주입에서 생성된 배아 수'] / train['미세주입된 난자 수'] * 100,
-999)
test['수정률'] = np.where(
    test['미세주입된 난자 수'] > 0,
    test['미세주입에서 생성된 배아 수'] / test['미세주입된 난자 수'] * 100,
-999)

# 각 행의 경과일 합계를 새로운 컬럼으로 생성
elapsed_days_columns = [
    '난자 채취 경과일',
    '난자 해동 경과일',
    '난자 혼합 경과일',
    '배아 이식 경과일',
    '배아 해동 경과일'
]

train['총 경과일 합계'] = train[elapsed_days_columns].sum(axis=1, numeric_only=True)
test['총 경과일 합계'] = test[elapsed_days_columns].sum(axis=1, numeric_only=True)

# 임신실패 횟수 관련 
train["총 임신 실패 횟수"] = train["총 임신 횟수"] - train["총 시술 횟수"]
test["총 임신 실패 횟수"] = test["총 임신 횟수"] - test["총 시술 횟수"]

train["IVF 임신 실패 횟수"] = train["IVF 임신 횟수"] - train["IVF 시술 횟수"]
test["IVF 임신 실패 횟수"] = test["IVF 임신 횟수"] - test["IVF 시술 횟수"]

train["DI 임신 실패 횟수"] = train["DI 임신 횟수"] - train["DI 시술 횟수"]
test["DI 임신 실패 횟수"] = test["DI 임신 횟수"] - test["DI 시술 횟수"]

# 유산 횟수 컬럼
train['총 유산 횟수'] = train['총 임신 횟수'] - train['총 출산 횟수']
train['IVF 유산 횟수'] = train['IVF 임신 횟수'] - train['IVF 출산 횟수']
train['DI 유산 횟수'] = train['DI 임신 횟수'] - train['DI 출산 횟수']

test['총 유산 횟수'] = test['총 임신 횟수'] - test['총 출산 횟수']
test['IVF 유산 횟수'] = test['IVF 임신 횟수'] - test['IVF 출산 횟수']
test['DI 유산 횟수'] = test['DI 임신 횟수'] - test['DI 출산 횟수']

# 난자와 정자의 기증자 나이에 따라 다를 수 있을 것이다.
train['기증자 나이 합산'] = train['난자 기증자 나이'] + train['정자 기증자 나이']
test['기증자 나이 합산'] = test['난자 기증자 나이'] + test['정자 기증자 나이']

# 미세 주입 성공률 -> 난자 미세주입 후 생성 된 배아수에 대한 정보
train['미세 주입 성공률'] = np.where(
    train['미세주입된 난자 수'] > 0,
    train['미세주입에서 생성된 배아 수'] / train['미세주입된 난자 수'] * 100,
    0
)
test['미세 주입 성공률'] = np.where(
    test['미세주입된 난자 수'] > 0,
    test['미세주입에서 생성된 배아 수'] / test['미세주입된 난자 수'] * 100,
    0
)

# 난자 채취 이후 경과일
train['난자 채취 이후 경과일'] = train['난자 채취 경과일'] + train['배아 이식 경과일']
test['난자 채취 이후 경과일'] = test['난자 채취 경과일'] + test['배아 이식 경과일']

# 난자 출처 + 정자 출처
train['배아 출처'] = train['난자 출처'].astype(str) + train['정자 출처'].astype(str)
test['배아 출처'] = test['난자 출처'].astype(str) + test['정자 출처'].astype(str)

where_mapping = {
    '11' : 1,
    '1-1' : 2,
    '-11' : 3,
    '-1-1' : 4,
    '1-999' : 5,
    '10' : 6,
    '-1-999': 7,
    '-10' : 8
}

train['배아 출처'] = train['배아 출처'].map(where_mapping)
test['배아 출처'] = test['배아 출처'].map(where_mapping)

# 특정 시술 유형 카테고리 리스트
categories = ['ICSI', 'IUI', 'ICI', 'GIFT', 'FER', 'GenericDI', 'IVI', 'BLASTOCYST', 'AH', 'Unknown', 'IVF', '-999', 'ICSI_ICSI', 'IVF_IVF', 'ICSI_Unknown', 'BLASTOCYST_IVF', 'IVF_Unknown', 'ICSI/BLASTOCYST', 'IVF/BLASTOCYST', 'ICSI/AH', 'IVF/AH', 'ICSI/BLASTOCYST_IVF/BLASTOCYST']

train['특정 시술 유형 (합산 유형)'] = train[[f'특시유_{cat}' for cat in categories]].sum(axis=1)
test['특정 시술 유형 (합산 유형)'] = test[[f'특시유_{cat}' for cat in categories]].sum(axis=1)

# 유전검사를 많이 했는지에 대한 여부를 합산으로 표현
col = [
'착상 전 유전 검사 사용 여부',
'착상 전 유전 진단 사용 여부',
'PGD 시술 여부',
'PGS 시술 여부'
]

train['유전검사 (합산통합)'] = train[col].sum(axis=1)
test['유전검사 (합산통합)'] = test[col].sum(axis=1)

# 불임원인이 많은지에 대한 여부를 합산으로 표현
col = [ '남성 주 불임 원인',
 '남성 부 불임 원인',
 '여성 주 불임 원인',
 '여성 부 불임 원인',
 '부부 주 불임 원인',
 '부부 부 불임 원인',
 '불명확 불임 원인',
 '불임 원인 - 난관 질환',
 '불임 원인 - 남성 요인',
 '불임 원인 - 배란 장애',
 '불임 원인 - 자궁경부 문제',
 '불임 원인 - 자궁내막증',
 '불임 원인 - 정자 농도',
 '불임 원인 - 정자 면역학적 요인',
 '불임 원인 - 정자 운동성',
 '불임 원인 - 정자 형태']

train['불임원인 (합산통합)'] = train[col].sum(axis=1)
test['불임원인 (합산통합)'] = test[col].sum(axis=1)

# 모든 남성 원인을 모아 한컬럼으로
def men_cause(data):

    cri = [
        data['남성 주 불임 원인'] == 1,
        data['남성 부 불임 원인'] == 1,
        data['불임 원인 - 정자 농도'] == 1,
        data['불임 원인 - 정자 면역학적 요인'] == 1,
        data['불임 원인 - 정자 운동성'] == 1,
        data['불임 원인 - 정자 형태'] == 1,
        data['불임 원인 - 남성 요인'] == 1,
    ]

    con = [
        1, 1, 1, 1, 1, 1, 1
    ]

    data['남성 불임 요인'] = np.select(cri, con, default = 0)

    return data
train = men_cause(train)
test = men_cause(test)

# 모든 여성 원인을 모아 한컬럼으로
def woman_cause(data):

    cri = [
        data['여성 주 불임 원인'] == 1,
        data['여성 부 불임 원인'] == 1,
        data['불임 원인 - 정자 농도'] == 1,
        data['불임 원인 - 난관 질환'] == 1,
        data['불임 원인 - 배란 장애'] == 1,
        data['불임 원인 - 자궁경부 문제'] == 1,
        data['불임 원인 - 자궁내막증'] == 1,
    ]

    con = [
        1, 1, 1, 1, 1, 1, 1
    ]

    data['여성 불임 요인'] = np.select(cri, con, default = 0)

    return data
train = woman_cause(train)
test = woman_cause(test)

# IVF와 ICSI에서 가장 확률이 높았던 조건들
def IVF_ICSI_best_condition(data):
    cri = [
        (data['특시유_IVF'] == 1) & (data['이식된 배아 수'] > 1) & (data['배아 이식 경과일'] == 5) & (data['해당 주기 난자 주인 나이'] == 6) & (data['배란 자극 여부'] == 1), 
        (data['특시유_ICSI'] == 1) & (data['이식된 배아 수'] > 1) & (data['배아 이식 경과일'] == 5) & (data['해당 주기 난자 주인 나이'] == 6) & (data['남성 불임 요인'] == 0)
    ]
    con = [
        1, 1
    ]
    data['IVF_ICSI_최상조건'] = np.select(cri, con, default=0)
    return data
train = IVF_ICSI_best_condition(train)
test = IVF_ICSI_best_condition(test)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
<ipython-input-19-ace09d6bddf5>:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train["IVF 임신 실패 횟수"] = train["IVF 임신 횟수"] - train["IVF 시술 횟수"]
<ipython-input-19-ace09d6bddf5>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test["IVF 임신 실패 횟수"] = test["IVF 임신 횟수"] - test["IVF 시술 횟수"]
<ipython-input-19-ace09d6bddf5>:67: PerformanceWarning: DataFrame is highly fragmented.  Th

## 5. Model Training
- 앙상블과 rule-based 로 Labeling을 진행하거나, LGBM, LOGISTIC REGRESSION을 사용
- 모델 분리  
    - DI  
    - IVF - 현재 시술용 - 이식 수 > 1 - MIXED (ICSI and IVF)
    - IVF - 현재 시술용 - 이식 수 > 1 - AH
    - IVF - 현재 시술용 - 이식 수 > 1 - BLASTOCYST
    - IVF - 현재 시술용 - 이식 수 > 1 - ALL

==========================================================  
- IVF - 현재 시술용 - 이식 수 == 0 -> 0처리 (현재 코드에서는 LGBM 단일 확률)  
- IVF - 현재 시술용 x -> 0처리 (현재 코드에서는 LGBM 단일 확률)  

In [20]:
# 모델에서 사용할 칼럼 선택하기
col = [
    '시술 시기 코드', '시술 유형', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', 
    '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인',
    '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인',
    '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
    
    '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
    '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 출처', '정자 출처',
    '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수',
    
    'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 
    'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수',
    '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
    
    '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
    '기증자 정자와 혼합된 난자 수', '난자 기증자 나이', '정자 기증자 나이', '난자 채취 경과일', '난자 해동 경과일',
    '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', 'PGS', 'PGD',
    
    'DI시술방법', 'IVF시술방법', '특시유_ICSI', '특시유_IUI', '특시유_ICI',
    '특시유_GIFT', '특시유_GenericDI', '특시유_IVI', '특시유_BLASTOCYST', '특시유_AH',
    '특시유_Unknown', '특시유_IVF', '특시유_-999', '특시유_ICSI_ICSI', '특시유_IVF_IVF',
    
    '특시유_ICSI_Unknown', '특시유_BLASTOCYST_IVF', '특시유_IVF_Unknown', '특시유_ICSI/BLASTOCYST', '특시유_IVF/BLASTOCYST',
    '특시유_ICSI/AH', '특시유_IVF/AH', '특시유_ICSI/BLASTOCYST_IVF/BLASTOCYST', '배생이_-999', '배생이_기증용',
    '배생이_난자 저장용', '배생이_배아 저장용', '배생이_연구용', '배생이_현재 시술용', '임신 경험 여부',
    
    '출산 경험 여부', '시술 경험 여부', '단일_배아_이식_여부_수정', '클리닉 외 시술 여부', '배아 출처',
    '특정 시술 유형 (합산 유형)', '유전검사 (합산통합)', '불임원인 (합산통합)', '남성 불임 요인', '여성 불임 요인',
    'IVF_ICSI_최상조건', 'IVF에서 생성된 배아 수', 'IVF 배아 이식 수', 'IVF 후 저장된 배아 수', '폐기된 배아 수',
    
    '미세주입 배아 실패 수', 'IVF 배아 실패 수', '해당 주기 난자 주인 나이', '나이_X_이식수', '배란자극_X_이식배아수',
    '잉여_배아_수', '이식률', '수정률', '총 경과일 합계', '총 임신 실패 횟수',
    'IVF 임신 실패 횟수', 'DI 임신 실패 횟수', '총 유산 횟수', 'IVF 유산 횟수', 'DI 유산 횟수',
    '기증자 나이 합산', '미세 주입 성공률', '난자 채취 이후 경과일'
]

train_ = train[col + ['임신 성공 여부']]
test_ = test[col]

### 5-1. DI

In [21]:
X_DI = train_.drop(columns=["임신 성공 여부", '나이_X_이식수', '배란자극_X_이식배아수', '기증자 나이 합산'])[train_['시술 유형'] == -100]
y_DI = train_[train_['시술 유형'] == -100]["임신 성공 여부"]
constant_columns = [col for col in X_DI.columns if X_DI[col].nunique() == 1]
X_DI = X_DI.drop(columns=constant_columns)
X_test_DI = test_[test_['시술 유형'] == -100].drop(columns=['나이_X_이식수', '배란자극_X_이식배아수', '기증자 나이 합산']).drop(columns=constant_columns)

X_DI = X_DI[['시술 시기 코드', '배란 자극 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',  '불임 원인 - 정자 농도',
       '불임 원인 - 정자 형태', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '정자 기증자 나이', '특시유_IUI',
       '특시유_ICI', '특시유_GIFT', '특시유_IVI']]

X_test_DI = X_test_DI[['시술 시기 코드', '배란 자극 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',  '불임 원인 - 정자 농도',
       '불임 원인 - 정자 형태', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '정자 기증자 나이', '특시유_IUI',
       '특시유_ICI', '특시유_GIFT', '특시유_IVI']]

categorical_columns_list_DI = [
'시술 시기 코드', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인',
 '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',
 '불임 원인 - 정자 농도', '불임 원인 - 정자 형태', '특시유_IUI', '특시유_ICI', '특시유_GIFT', '특시유_IVI',
]

In [22]:
best_params = {'n_estimators': 14, 'max_depth': 6, 'learning_rate': 0.8437303848611053, 'colsample_bytree': 0.4475283388572317, 'min_child_weight': 76}

xgb_best_model_DI = XGBClassifier(
    **best_params,
    random_state=42,
    verbosity=0,
)

xgb_best_model_DI.fit(X_DI, y_DI)
xgb_pred_DI = xgb_best_model_DI.predict_proba(X_test_DI)[:, 1]
xgb_train_DI = xgb_best_model_DI.predict_proba(X_DI)[:, 1]

In [23]:
best_params = {'n_estimators': 10, 'learning_rate': 0.8535922551635277, 'feature_fraction': 0.4146990127204148, 'min_data_in_leaf': 546, 'num_leaves': 58, 'bagging_freq': 1}

lgbm_best_model_DI = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_DI.fit(X_DI, y_DI, categorical_feature=categorical_columns_list_DI)

lgbm_pred_DI = lgbm_best_model_DI.predict_proba(X_test_DI)[:, 1]
lgbm_train_DI = lgbm_best_model_DI.predict_proba(X_DI)[:, 1]

In [24]:
train_pool = Pool(X_DI, y_DI, cat_features=categorical_columns_list_DI)
test_pool = Pool(X_test_DI, cat_features=categorical_columns_list_DI)

best_params = {'iterations': 28, 'depth': 3, 'learning_rate': 0.6779854308369335, 'rsm': 0.7858274950839619, 'min_data_in_leaf': 352}
cat_best_model_DI = CatBoostClassifier(
    **best_params,
    verbose=0,
    random_seed=42,
    eval_metric="Logloss",
)

cat_best_model_DI.fit(      
        train_pool,
        verbose=0,
    )

cat_pred_DI = cat_best_model_DI.predict_proba(test_pool)[:, 1]
cat_train_DI = cat_best_model_DI.predict_proba(train_pool)[:, 1]

In [25]:
# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (1, 1, 1)
ens_test_DI = (a * lgbm_pred_DI + b * xgb_pred_DI + c *  cat_pred_DI) / (a + b + c)
test_.loc[test_['시술 유형'] == -100, 'probability'] = ens_test_DI

# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (1, 1, 1)
ens_train_DI = (a * lgbm_train_DI + b * xgb_train_DI + c *  cat_train_DI) / (a + b + c)
train.loc[train['시술 유형'] == -100, 'probability'] = ens_train_DI

<ipython-input-25-d8f8faba3a66>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_.loc[test_['시술 유형'] == -100, 'probability'] = ens_test_DI
<ipython-input-25-d8f8faba3a66>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_.loc[test_['시술 유형'] == -100, 'probability'] = ens_test_DI
<ipython-input-25-d8f8faba3a66>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

### 5-2. IVF 현재 시술용인 경우

In [26]:
X_IVF_NOW_YES = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0].drop(columns=['DI시술방법'])
y_IVF_NOW_YES = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0]["임신 성공 여부"]
constant_columns = [col for col in X_IVF_NOW_YES.columns if X_IVF_NOW_YES[col].nunique() == 1]
X_IVF_NOW_YES = X_IVF_NOW_YES.drop(columns=constant_columns)

X_test_IVF_NOW_YES = test_[test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 1][test_['이식된 배아 수'] != 0].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')

categorical_columns_list_IVF_NOW_YES = [
    'PGS', 'PGD', '특시유_ICSI', '특시유_BLASTOCYST', '특시유_AH', '특시유_Unknown', '특시유_IVF', '특시유_ICSI_ICSI', '특시유_IVF_IVF', '특시유_ICSI_Unknown', '특시유_BLASTOCYST_IVF',
    '특시유_IVF_Unknown', '배생이_기증용', '배생이_난자 저장용', '배생이_배아 저장용', '임신 경험 여부', '출산 경험 여부', '시술 경험 여부', '클리닉 외 시술 여부', '배아 출처', '특정 시술 유형 (합산 유형)', '불임원인 (합산통합)',
    '남성 불임 요인', '여성 불임 요인', 'IVF_ICSI_최상조건', '시술 시기 코드', '배란 자극 여부', '단일 배아 이식 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인',
    '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', 
    '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 출처', '정자 출처'
]

<ipython-input-26-2bff6e6e2203>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_IVF_NOW_YES = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0].drop(columns=['DI시술방법'])
<ipython-input-26-2bff6e6e2203>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_IVF_NOW_YES = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0].drop(columns=['DI시술방법'])
<ipython-input-26-2bff6e6e2203>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_IVF_NOW_YES = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0]["임신 성공 여부"]
<ipython-input-26-2bff6e6e2203>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_IVF_NOW_YES = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0]["임신 성공 여부"]
<ipython-input-26-2bff6e

In [27]:
best_params = {'n_estimators': 383, 'max_depth': 4, 'learning_rate': 0.09073667416758903, 'colsample_bytree': 0.4391667088663101, 'min_child_weight': 327}
xgb_best_model_IVF = XGBClassifier(
    **best_params,
    random_state=42,
    verbosity=0,
)

xgb_best_model_IVF.fit(X_IVF_NOW_YES, y_IVF_NOW_YES)
xgb_pred_IVF_NOW_YES = xgb_best_model_IVF.predict_proba(X_test_IVF_NOW_YES)[:, 1]
xgb_train_IVF_NOW_YES = xgb_best_model_IVF.predict_proba(X_IVF_NOW_YES)[:, 1]

In [28]:
best_params = {'n_estimators': 297, 'learning_rate': 0.027999291772526146, 'feature_fraction': 0.3844858949031732, 'min_data_in_leaf': 714, 'num_leaves': 44, 'bagging_freq': 79, 'bagging_fraction': 0.8726523819009622, 'max_bin': 7, 'min_child_weight': 0.039415245205013445}
lgbm_best_model_IVF = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_IVF.fit(X_IVF_NOW_YES, y_IVF_NOW_YES, categorical_feature=categorical_columns_list_IVF_NOW_YES)

lgbm_pred_IVF_NOW_YES = lgbm_best_model_IVF.predict_proba(X_test_IVF_NOW_YES)[:, 1]
lgbm_train_IVF_NOW_YES = lgbm_best_model_IVF.predict_proba(X_IVF_NOW_YES)[:, 1]

In [29]:
train_pool = Pool(X_IVF_NOW_YES, y_IVF_NOW_YES, cat_features=categorical_columns_list_IVF_NOW_YES) 
test_pool = Pool(X_test_IVF_NOW_YES, cat_features=categorical_columns_list_IVF_NOW_YES)

best_params =  {'learning_rate': 0.030695032701723943, 'rsm': 0.6755616862306184, 'min_data_in_leaf': 269, 'border_count': 6, 'one_hot_max_size': 12}
cat_best_model_IVF = CatBoostClassifier(
    **best_params,
    verbose=0,
    random_seed=42,
    eval_metric="Logloss",
    thread_count=4
)

cat_best_model_IVF.fit(      
        train_pool,
        verbose=0,
)

cat_pred_IVF_NOW_YES = cat_best_model_IVF.predict_proba(test_pool)[:, 1]
cat_train_IVF_NOW_YES = cat_best_model_IVF.predict_proba(train_pool)[:, 1]

In [30]:
# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (13, 9, 10)
ens_test_IVF_NOW_YES = (a * lgbm_pred_IVF_NOW_YES + b * xgb_pred_IVF_NOW_YES + c *  cat_pred_IVF_NOW_YES) / (a + b + c)
test_.loc[(test_['시술 유형'] == 100) & (test_['배생이_현재 시술용'] == 1) & (test_['이식된 배아 수'] != 0), 'probability'] = ens_test_IVF_NOW_YES

# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (13, 9, 10)
ens_train_IVF_NOW_YES = (a * lgbm_train_IVF_NOW_YES + b * xgb_train_IVF_NOW_YES + c *  cat_train_IVF_NOW_YES) / (a + b + c)
train.loc[(train['시술 유형'] == 100) & (train['배생이_현재 시술용'] == 1) & (train['이식된 배아 수'] != 0), 'probability'] = ens_train_IVF_NOW_YES

### 5-3. 이식된 배아 수가 없는 경우
- 당연히 임신할 수가 없다. -> 전부 0 혹은 LOGISTIC / LGBM

In [31]:
X_IVF_NOW_NO = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] == 0].drop(columns=['DI시술방법'])
y_IVF_NOW_NO = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] == 0]["임신 성공 여부"]
constant_columns = [col for col in X_IVF_NOW_NO.columns if X_IVF_NOW_NO[col].nunique() == 1]
X_IVF_NOW_NO = X_IVF_NOW_NO.drop(columns=constant_columns)

X_test_IVF_NOW_NO = test_[test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 1][test_['이식된 배아 수'] == 0].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')

<ipython-input-31-d4040318bba1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_IVF_NOW_NO = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] == 0].drop(columns=['DI시술방법'])
<ipython-input-31-d4040318bba1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_IVF_NOW_NO = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] == 0].drop(columns=['DI시술방법'])
<ipython-input-31-d4040318bba1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_IVF_NOW_NO = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] == 0]["임신 성공 여부"]
<ipython-input-31-d4040318bba1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_IVF_NOW_NO = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] == 0]["임신 성공 여부"]
<ipython-input-31-d4040318bb

In [32]:
best_params = {'n_estimators': 19, 'learning_rate': 0.6274517807598488, 'feature_fraction': 0.22284237439031715, 'min_data_in_leaf': 470, 'num_leaves': 29, 'max_bin': 3, 'bagging_freq': 12, 'bagging_fraction': 0.7150190897915791}
lgbm_best_model_IVF_NOW_NO = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_IVF_NOW_NO.fit(X_IVF_NOW_NO, y_IVF_NOW_NO)

lgbm_pred_IVF_NOW_NO = lgbm_best_model_IVF_NOW_NO.predict_proba(X_test_IVF_NOW_NO)[:, 1]
lgbm_train_IVF_NOW_NO = lgbm_best_model_IVF_NOW_NO.predict_proba(X_IVF_NOW_NO)[:, 1]

In [33]:
test_.loc[(test_['시술 유형'] == 100) & (test_['배생이_현재 시술용'] == 1) & (test_['이식된 배아 수'] == 0), 'probability'] = lgbm_pred_IVF_NOW_NO
train.loc[(train['시술 유형'] == 100) & (train['배생이_현재 시술용'] == 1) & (train['이식된 배아 수'] == 0), 'probability'] = lgbm_train_IVF_NOW_NO

### 5-4. IVF 현재 시술용 아닌 경우
- 임신의 용도가 아니다. -> 전부 0 혹은 LOGISTIC / LGBM

In [34]:
X_IVF_NOTNOW = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 0].drop(columns=['DI시술방법'])
y_IVF_NOTNOW = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 0]["임신 성공 여부"]
constant_columns = [col for col in X_IVF_NOTNOW.columns if X_IVF_NOTNOW[col].nunique() == 1]
X_IVF_NOTNOW = X_IVF_NOTNOW.drop(columns=constant_columns)

X_test_IVF_NOTNOW = test_[test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 0].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')

<ipython-input-34-615eddb60c09>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_IVF_NOTNOW = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 0].drop(columns=['DI시술방법'])
<ipython-input-34-615eddb60c09>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_IVF_NOTNOW = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 0]["임신 성공 여부"]
<ipython-input-34-615eddb60c09>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_test_IVF_NOTNOW = test_[test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 0].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')


In [35]:
best_params = {'n_estimators': 11, 'learning_rate': 0.6427329093577366, 'feature_fraction': 0.266606084905757, 'min_data_in_leaf': 636, 'num_leaves': 93, 'max_bin': 6, 'bagging_freq': 89, 'bagging_fraction': 0.2800177699349053} # {'n_estimators': 14, 'learning_rate': 0.15192316905303452, 'feature_fraction': 0.8941396223112753, 'min_data_in_leaf': 2464, 'num_leaves': 46}
lgbm_best_model_IVF_NOTNOW = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_IVF_NOTNOW.fit(X_IVF_NOTNOW, y_IVF_NOTNOW)

lgbm_pred_IVF_NOTNOW = lgbm_best_model_IVF_NOTNOW.predict_proba(X_test_IVF_NOTNOW)[:, 1]
lgbm_train_IVF_NOTNOW = lgbm_best_model_IVF_NOTNOW.predict_proba(X_IVF_NOTNOW)[:, 1]

In [36]:
test_.loc[(test_['시술 유형'] == 100) & (test_['배생이_현재 시술용'] == 0), 'probability'] = lgbm_pred_IVF_NOTNOW
train.loc[(train['시술 유형'] == 100) & (train['배생이_현재 시술용'] == 0), 'probability'] = lgbm_train_IVF_NOTNOW

### 5-5.MIXED

In [37]:
mixed_list = ["ICSI_IVF", "IVF_ICSI", "ICSI/BLASTOCYST_IVF/BLASTOCYST", "IVF/AH_ICSI/AH"]
X_MIXED = train_[train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법'])[train['특정 시술 유형'].isin(mixed_list)]
y_MIXED = train_[train_['이식된 배아 수'] != 0][train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train['특정 시술 유형'].isin(mixed_list)]["임신 성공 여부"]
constant_columns = [col for col in X_MIXED.columns if X_MIXED[col].nunique() == 1]

X_MIXED = X_MIXED.drop(columns=constant_columns)
X_test_MIXED = test_[test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 1][test_['이식된 배아 수'] != 0].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')[test['특정 시술 유형'].isin(mixed_list)]

<ipython-input-37-766dea61be03>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_MIXED = train_[train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법'])[train['특정 시술 유형'].isin(mixed_list)]
<ipython-input-37-766dea61be03>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_MIXED = train_[train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법'])[train['특정 시술 유형'].isin(mixed_list)]
<ipython-input-37-766dea61be03>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_MIXED = train_[train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법'])[train['특정 시술 유형'].isin(mixed_list)]
<ipython-input-37-766dea61be03>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_MIX

In [38]:
best_params = {'n_estimators': 5, 'learning_rate': 0.46500212903895416, 'feature_fraction': 0.47911362428539434, 'min_data_in_leaf': 185, 'num_leaves': 2}
lgbm_best_model_MIXED = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_MIXED.fit(X_MIXED, y_MIXED)

lgbm_pred_MIXED = lgbm_best_model_MIXED.predict_proba(X_test_MIXED)[:, 1]
lgbm_train_MIXED = lgbm_best_model_MIXED.predict_proba(X_MIXED)[:, 1]

In [39]:
test_.loc[(test['특정 시술 유형'].isin(mixed_list)) & (test_['배생이_현재 시술용'] == 1) & (test_['이식된 배아 수'] != 0), 'probability'] = lgbm_pred_MIXED
train_.loc[(train['특정 시술 유형'].isin(mixed_list)) & (train_['배생이_현재 시술용'] == 1) & (train_['이식된 배아 수'] != 0), 'probability'] = lgbm_train_MIXED

<ipython-input-39-e7a4ad119ff9>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_.loc[(train['특정 시술 유형'].isin(mixed_list)) & (train_['배생이_현재 시술용'] == 1) & (train_['이식된 배아 수'] != 0), 'probability'] = lgbm_train_MIXED
<ipython-input-39-e7a4ad119ff9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_.loc[(train['특정 시술 유형'].isin(mixed_list)) & (train_['배생이_현재 시술용'] == 1) & (train_['이식된 배아 수'] != 0), 'probability'] = lgbm_train_MIXED


### 5-6. BLASTOCYST

In [40]:
X_BLASTOCYST = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0][train_['특시유_BLASTOCYST'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
y_BLASTOCYST = train_[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0][train_['특시유_BLASTOCYST'] == 1]['임신 성공 여부']
constant_columns = [col for col in X_BLASTOCYST.columns if X_BLASTOCYST[col].nunique() == 1]
X_BLASTOCYST = X_BLASTOCYST.drop(columns=constant_columns)
X_test_BLASTOCYST = test_[test_['특시유_BLASTOCYST'] == 1][test_['이식된 배아 수'] != 0][test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 1].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')

<ipython-input-40-c5511c054a62>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_BLASTOCYST = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0][train_['특시유_BLASTOCYST'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
<ipython-input-40-c5511c054a62>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_BLASTOCYST = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0][train_['특시유_BLASTOCYST'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
<ipython-input-40-c5511c054a62>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_BLASTOCYST = train_.drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1][train_['이식된 배아 수'] != 0][train_['특시유_BLASTOCYST'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
<ipython-input-40-c5511c054a62>:2:

In [41]:
best_params = {'n_estimators': 47, 'max_depth': 4, 'min_child_weight': 214, 'subsample': 0.8720615036283587, 'colsample_bynode': 0.6446105555057425, 'learning_rate': 0.8878810016000835, 'colsample_bytree': 0.917469599745884}

xgb_best_model_BLASTOCYST = XGBClassifier(
    **best_params,
    random_state=42,
    verbosity=0,
)

xgb_best_model_BLASTOCYST.fit(X_BLASTOCYST, y_BLASTOCYST)
xgb_pred_BLASTOCYST = xgb_best_model_BLASTOCYST.predict_proba(X_test_BLASTOCYST)[:, 1]
xgb_train_BLASTOCYST = xgb_best_model_BLASTOCYST.predict_proba(X_BLASTOCYST)[:, 1]

In [42]:
best_params = {'n_estimators': 2, 'learning_rate': 0.568446956023203, 'feature_fraction': 0.3351344143816874, 'min_data_in_leaf': 49, 'num_leaves': 21, 'bagging_freq': 2}
lgbm_best_model_BLASTOCYST = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_BLASTOCYST.fit(X_BLASTOCYST, y_BLASTOCYST)

lgbm_pred_BLASTOCYST = lgbm_best_model_BLASTOCYST.predict_proba(X_test_BLASTOCYST)[:, 1]
lgbm_train_BLASTOCYST = lgbm_best_model_BLASTOCYST.predict_proba(X_BLASTOCYST)[:, 1]

In [43]:
train_pool = Pool(X_BLASTOCYST, y_BLASTOCYST)
test_pool = Pool(X_test_BLASTOCYST)

best_params =  {'iterations': 50, 'depth': 1, 'learning_rate': 0.9995518350903149, 'min_data_in_leaf': 973, 'rsm': 0.7009250500733488, 'border_count': 3, 'one_hot_max_size': 43}
cat_best_model_BLASTOCYST = CatBoostClassifier(
    **best_params,
    verbose=0,
    random_seed=42,
)

cat_best_model_BLASTOCYST.fit(      
        train_pool,
        verbose=0,
    )

cat_pred_BLASTOCYST = cat_best_model_BLASTOCYST.predict_proba(test_pool)[:, 1]
cat_train_BLASTOCYST = cat_best_model_BLASTOCYST.predict_proba(train_pool)[:, 1]

In [44]:
# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (1, 1, 1)
ens_test_BLASTOCYST = (a * lgbm_pred_BLASTOCYST + b * xgb_pred_BLASTOCYST + c *  cat_pred_BLASTOCYST) / (a + b + c)
test_.loc[(test_['특시유_BLASTOCYST'] == 1) & (test_['배생이_현재 시술용'] == 1) & (test_['이식된 배아 수'] != 0), 'probability'] = ens_test_BLASTOCYST

# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (1, 1, 1)
ens_train_BLASTOCYST = (a * lgbm_train_BLASTOCYST + b * xgb_train_BLASTOCYST + c *  cat_train_BLASTOCYST) / (a + b + c)
train.loc[(train['특시유_BLASTOCYST'] == 1) & (train['배생이_현재 시술용'] == 1) & (train['이식된 배아 수'] != 0), 'probability'] = ens_train_BLASTOCYST

### 5-7. AH

In [45]:
X_AH = train_[train_['특시유_AH'] == 1][train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
y_AH = train_[train_['특시유_AH'] == 1][train_['이식된 배아 수'] != 0][train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1]["임신 성공 여부"]
constant_columns = [col for col in X_AH.columns if X_AH[col].nunique() == 1]

X_AH = X_AH.drop(columns=constant_columns)
X_test_AH = test_[test_['시술 유형'] == 100][test_['배생이_현재 시술용'] == 1][test_['특시유_AH'] == 1][test_['이식된 배아 수'] != 0].drop(columns=constant_columns).drop(columns=['DI시술방법']).drop(columns='probability')

<ipython-input-45-f4a8d32a10af>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_AH = train_[train_['특시유_AH'] == 1][train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
<ipython-input-45-f4a8d32a10af>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_AH = train_[train_['특시유_AH'] == 1][train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
<ipython-input-45-f4a8d32a10af>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_AH = train_[train_['특시유_AH'] == 1][train_['이식된 배아 수'] != 0].drop(columns=["임신 성공 여부"])[train_['시술 유형'] == 100][train_['배생이_현재 시술용'] == 1].drop(columns=['DI시술방법']).drop(columns='probability')
<ipython-input-45-f4a8d32a10af>:2: UserWarning: Boolean Series key will be reindex

In [46]:
best_params = {'n_estimators': 16, 'max_depth': 1, 'min_child_weight': 18, 'subsample': 0.9665626084699487, 'colsample_bynode': 0.8749543188162278, 'learning_rate': 0.8422386595268349, 'colsample_bytree': 0.3635217907528703}

xgb_best_model_AH = XGBClassifier(
    **best_params,
    random_state=42,
    verbosity=0,
)

xgb_best_model_AH.fit(X_AH, y_AH)
xgb_pred_AH = xgb_best_model_AH.predict_proba(X_test_AH)[:, 1]
xgb_train_AH = xgb_best_model_AH.predict_proba(X_AH)[:, 1]

In [47]:
best_params = {'n_estimators': 2, 'learning_rate': 0.35837002632562415, 'feature_fraction': 0.6533029668795871, 'min_data_in_leaf': 87, 'num_leaves': 42, 'bagging_freq': 4}
lgbm_best_model_AH = LGBMClassifier(**best_params, random_state=42, verbose=-1)
lgbm_best_model_AH.fit(X_AH, y_AH)

lgbm_pred_AH = lgbm_best_model_AH.predict_proba(X_test_AH)[:, 1]
lgbm_train_AH = lgbm_best_model_AH.predict_proba(X_AH)[:, 1]

In [48]:
train_pool = Pool(X_AH, y_AH)
test_pool = Pool(X_test_AH)

best_params =  {'iterations': 9, 'depth': 2, 'learning_rate': 0.9946932992610313, 'rsm': 0.9065280177387596, 'min_data_in_leaf': 15, 'border_count': 131, 'one_hot_max_size': 9}
cat_best_model_AH = CatBoostClassifier(
    **best_params,
    verbose=0,
    random_seed=42,
    eval_metric="Logloss",
)

cat_best_model_AH.fit(      
        train_pool,
        verbose=0,
)

cat_pred_AH = cat_best_model_AH.predict_proba(test_pool)[:, 1]
cat_train_AH = cat_best_model_AH.predict_proba(train_pool)[:, 1]

In [49]:
# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (1, 1, 1)
ens_test_AH = (a * lgbm_pred_AH + b * xgb_pred_AH + c * cat_pred_AH) / (a + b + c)
test_.loc[(test_['특시유_AH'] == 1) & (test_['배생이_현재 시술용'] == 1) & (test_['이식된 배아 수'] != 0), 'probability'] = ens_test_AH

# 세 모델의 확률 Soft Voting Ensemble
a, b, c = (1, 1, 1)
ens_train_AH = (a * lgbm_train_AH + b * xgb_train_AH + c *  cat_train_AH) / (a + b + c)
train.loc[(train['특시유_AH'] == 1) & (train['배생이_현재 시술용'] == 1) & (train['이식된 배아 수'] != 0), 'probability'] = ens_train_AH

### 6.Inference
- 제출물 test를 받아 확률을 예측하고 제출 파일을 생성한다.
- Train에 대한 확률에 대해서 IsotonicRegression을 학습시켜, test의 probability를 Calibration 한다.

In [ ]:
import numpy as np
from sklearn.isotonic import IsotonicRegression

# Calibration을 진행함. Train의 확률을 바탕으로, 예측되어진 Test의 확률을 보정함.
# Train중 절반의 데이터셋만 샘플링하여, isotonic Regression을 진행
rng = np.random.default_rng(2002) # 대한민국의 4강신화는 다시올수있을까

train_prob = np.array(train['probability'])
train_labels = np.array(train['임신 성공 여부'])

sample_size = int(0.5 * len(train_prob))
sample_indices = rng.choice(len(train_prob), size=sample_size, replace=False)
train_prob_sampled = train_prob[sample_indices]
train_labels_sampled = train_labels[sample_indices]

iso_reg = IsotonicRegression(out_of_bounds='clip')
iso_reg.fit(train_prob_sampled, train_labels_sampled)

train_prob_calibrated = iso_reg.transform(train_prob)
train.loc[:, 'probability_calibrated'] = train_prob_calibrated

test_prob = np.array(test_['probability'])
test_prob_calibrated = iso_reg.transform(test_prob)
test_.loc[:, 'probability_calibrated'] = test_prob_calibrated

<ipython-input-50-64a2491b5f4f>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train.loc[:, 'probability_calibrated'] = train_prob_calibrated
<ipython-input-50-64a2491b5f4f>:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_.loc[:, 'probability_calibrated'] = test_prob_calibrated
<ipython-input-50-64a2491b5f4f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [ ]:
df_sub["probability"] = test_['probability_calibrated']
df_sub.to_csv("submission.csv", index=False)
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           90067 non-null  object 
 1   probability  90067 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


### A.검증용 성적계산
- 하단 점수가 같으면, 같은 결과가 나왔다고 볼 수 있습니다.

In [55]:
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    make_scorer,
    roc_curve,
    auc,
    precision_recall_curve,
    roc_auc_score,
    silhouette_score
)

roc_auc_score(train['임신 성공 여부'], train['probability']) # 0.7514802887078333

0.7514802887078333

In [56]:
roc_auc_score(train[train['시술 유형'] == -100]['임신 성공 여부'], train[train['시술 유형'] == -100]['probability_calibrated']) # 0.7310932254493416

0.7310932254493416

In [57]:
roc_auc_score(train[train['시술 유형'] == 100]['임신 성공 여부'], train[train['시술 유형'] == 100]['probability_calibrated']) # 0.7496462189217733

0.7496462189217733

- (이외에 다른점이 있던 라이브러리 목록입니다. 검증이 틀려지게 된다면, 하단 라이브러리까지 고려해주시길 바랍니다.)
- fonttools==4.55.3
- kiwisolver==1.4.7
- narwhals==1.18.4
- pillow==11.0.0
- plotly==5.24.1
- types-python-dateutil==2.9.0.20241206
- types-pytz==2024.2.0.20241003
- tzdata==2024.2
- pip install fonttools==4.55.3 kiwisolver==1.4.7 narwhals==1.18.4 pillow==11.0.0 plotly==5.24.1 types-python-dateutil==2.9.0.20241206 types-pytz==2024.2.0.20241003 tzdata==2024.2
